In [1]:
import numpy as np 

import pandas as pd
from pandas_datareader import data as web 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates 
%matplotlib inline

import datetime as dt 
import mplfinance as mpf 

import time

import os
from os import listdir
from os.path import isfile, join

In [2]:
# Default Values
PATH = "F:/Code/Python/Jupyter/Finance/datasets/Stock List/"
S_YEAR = 2017
S_MONTH = 1
S_DAY = 3
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

E_YEAR = 2021
E_MONTH = 8
E_DAY = 19
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

In [3]:
# Get Stock File Names in a List
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]

tickers.sort()

In [4]:
# Function that returns a dataframe from a csv
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + ".csv")
    except FileNotFoundError:
        print("File doesn't exist")
    else:
        return df

In [5]:
# Function that saves dataframe to csv
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + ".csv")

In [6]:
# Return valid dates in dataframe
def get_valid_dates(df, sdate, edate):
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate)
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        
        date_leading = '-'.join(('0' if len(x) < 2 else '')+ x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x) < 2 else '')+ x for x in last_date.split('-'))
    except Exception:
        print("Date is corrupted")
    else:
        return date_leading, date_ending

In [7]:
# Returns return on investment over specific time
def roi_between_dates(df, sdate, edate):
    try:
        start_val = df.loc[sdate, 'Adj Close']
        end_val = df.loc[edate, 'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data is corrupted")
    else:
        return roi

In [8]:
# Get mean between dates
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    
    return df.loc[mask]['Adj Close'].mean()

In [9]:
# Get standard deviation between dates
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    
    return df.loc[mask]['Adj Close'].std()

In [10]:
# Get coefficient of variation between dates
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    
    return sd / mean

In [11]:
# Merge multiple stocks in one dataframe by a specific column name
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
    
    return mult_df

In [12]:
# Get Sector Stocks
sec_df = pd.read_csv("F:/Code/Python/Jupyter/Finance/datasets/stock_sectors.csv")
sec_df

indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

In [13]:
# Returns a dataframe with ROIs for all stocks
def get_rois_for_stocks(stock_df):
    tickers = []
    rois = []
    
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
        if df is None:
            pass
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2018-01-01', '2020-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)

    return pd.DataFrame({'Ticker': tickers, 'ROI': rois})

In [ ]:
industrial = get_rois_for_stocks(indus_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)

In [15]:
# Get top industrial stocks based on ROI
industrial.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
6,GNRC,3.734749
4,AME,0.698376
2,ALLE,0.504415
0,AOS,-0.060680
5,BA,-0.243301


In [16]:
# Get top health care stocks based on ROI
health_care.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
15,DXCM,5.375582
10,BIO,1.414630
4,ALGN,1.379040
0,ABT,0.958374
3,A,0.795687


In [17]:
# Get top IT stocks based on ROI
it.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
2,AMD,7.352459
7,AAPL,2.201818
10,ADSK,1.850448
1,ADBE,1.814406
6,ANSS,1.449007


In [18]:
# Get top communications stocks based on ROI
commun.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
3,NFLX,1.689262
1,GOOG,0.644958
2,FB,0.505677
0,ATVI,0.472510


In [19]:
# Get top staple stocks based on ROI
staple.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
2,COST,1.112863
0,ADM,0.385834
1,CAG,0.047754


In [20]:
# Get top discretionary stocks based on ROI
discretion.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
7,TGT,1.839621
1,AMZN,1.739195
3,AZO,0.609471
4,BBY,0.563621
2,APTV,0.548157


In [21]:
# Get top utilities stocks based on ROI
utility.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
0,AES,1.402798
3,AWK,0.819223
1,AEE,0.450551
2,AEP,0.271589
4,ATO,0.198211


In [22]:
# Get top finance stocks based on ROI
finance.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
13,MSCI,2.612068
6,AJG,1.112315
5,AON,0.659334
9,BLK,0.536003
7,AIZ,0.466652


In [23]:
# Get top materials stocks based on ROI
material.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
3,BLL,1.474667
0,APD,0.771643
4,NEM,0.696606
2,AVY,0.421215
1,ALB,0.180301


In [24]:
# Get top real estate stocks based on ROI
restate.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
4,SBAC,0.772695
1,AMT,0.685783
0,ARE,0.492708
2,AVB,0.005870
3,BXP,-0.189743


In [25]:
# Get top energy stocks based on ROI
energy.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
1,HES,0.168440
0,APA,-0.654885


In [42]:
# Simulate portfolio list
port_list = ["GNRC", "DXCM", "AMD", "NFLX", "COST", "TGT", "AES", "MSCI", "NEM", "SBAC", "HES"]


In [27]:
mult_df = merge_df_by_column_name('daily_return', '2018-01-01', '2020-12-31', *port_list)
mult_df.corr()

,GNRC,DXCM,AMD,NFLX,COST,TGT,AES,MSCI,NEM,SBAC,HES
GNRC,1.000000,0.289526,0.395187,0.340732,0.465303,0.338411,0.433322,0.515742,0.250839,0.444762,0.426870
DXCM,0.289526,1.000000,0.328297,0.307643,0.265366,0.196806,0.218426,0.469509,0.147613,0.243462,0.243240
AMD,0.395187,0.328297,1.000000,0.459581,0.394313,0.340573,0.258976,0.485819,0.191451,0.294906,0.288337
NFLX,0.340732,0.307643,0.459581,1.000000,0.430674,0.259971,0.204163,0.449132,0.103912,0.216935,0.217857
COST,0.465303,0.265366,0.394313,0.430674,1.000000,0.500433,0.319887,0.482323,0.183703,0.455022,0.234526
TGT,0.338411,0.196806,0.340573,0.259971,0.500433,1.000000,0.305005,0.325771,0.092807,0.304364,0.245106
AES,0.433322,0.218426,0.258976,0.204163,0.319887,0.305005,1.000000,0.483397,0.176953,0.333249,0.470413
MSCI,0.515742,0.469509,0.485819,0.449132,0.482323,0.325771,0.483397,1.000000,0.258856,0.498281,0.411647
NEM,0.250839,0.147613,0.191451,0.103912,0.183703,0.092807,0.176953,0.258856,1.000000,0.275892,0.149227
SBAC,0.444762,0.243462,0.294906,0.216935,0.455022,0.304364,0.333249,0.498281,0.275892,1.000000,0.273548


In [28]:
days = len(mult_df.index)
days

756

In [29]:
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    
    return df.loc[date, 'Adj Close']

In [43]:
# Get stock price on the 4th of Jan, 2021 (first trading day of 2021) of the stocks in the simulated portfolio list

print("GNRC $", get_stock_price_on_date('GNRC', '2021-01-04')) # 2 449.12
print("DXCM $", get_stock_price_on_date('DXCM', '2021-01-04')) # 1 357.74
print("AMD $", get_stock_price_on_date('AMD', '2021-01-04')) # 4 369.2
print("NFLX $", get_stock_price_on_date('NFLX', '2021-01-04')) # 1 522.86
print("COST $", get_stock_price_on_date('COST', '2021-01-04')) # 1 377.88
print("TGT $", get_stock_price_on_date('TGT', '2021-01-04')) # 3 527.46
print("AES $", get_stock_price_on_date('AES', '2021-01-04')) # 18 410.76
print("MSCI $", get_stock_price_on_date('MSCI', '2021-01-04')) # 1 435.40
print("NEM $", get_stock_price_on_date('NEM', '2021-01-04')) # 7 434.49
print("SBAC $", get_stock_price_on_date('SBAC', '2021-01-04')) # 2 548.84
print("HES $", get_stock_price_on_date('HES', '2021-01-04')) # 9 472.23

GNRC $ 224.5599975585937
DXCM $ 357.739990234375
AMD $ 92.3000030517578
NFLX $ 522.8599853515625
COST $ 376.7002258300781
TGT $ 174.46778869628906
AES $ 22.522554397583008
MSCI $ 433.8714294433594
NEM $ 60.45637512207031
SBAC $ 272.9176940917969
HES $ 51.91611862182617


In [31]:
# Returns Portfolio Weight
def get_port_weight(price, total):
    return price / total

In [41]:
# Get weights to calculate covariance
gnrc_w = get_port_weight(449.12, 4905.98)
dxcm_w = get_port_weight(357.74, 4905.98)
amd_w = get_port_weight(369.2, 4905.98)
nflx_w = get_port_weight(522.86, 4905.98)
cost_w = get_port_weight(377.88, 4905.98)
tgt_w = get_port_weight(527.46, 4905.98)
aes_w = get_port_weight(410.76, 4905.98)
msci_w = get_port_weight(435.40, 4905.98)
nem_w = get_port_weight(434.49, 4905.98)
sbac_w = get_port_weight(548.84, 4905.98)
hes_w = get_port_weight(472.23, 4905.98)

wts = np.array([gnrc_w, dxcm_w, amd_w, nflx_w, cost_w, tgt_w, aes_w, msci_w,nem_w, sbac_w, hes_w])

print("Portfolio Covariance :", np.dot(wts.T, np.dot(mult_df.cov() * 756, wts)))

Portfolio Covariance : 0.1822108916606152


<h3>Different types of risk</h3>
<p>
    Idiosyncratic - Able to limit risk through diversification<br>
    
</p>

<p>
    Systematic - Not able to limit risk through diversification. Systematic risk is caused by unforeseen conditions such as wars, recessions, natural disasters, dramatic changes in consumer actions, etc.
</p>

<p>
    We can dramatically lower idiosyncratic risk by making a portfolio of approx. 25 stocks that aren't correlated according to data. 
</p>

<p>Diversifiable Risk = Portfolio Variance - All Weighted Variances</p>

In [45]:
def calc_diversifiable_risk(df, tickers, weights):
    days = len(mult_df.index)
    # Covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [34]:
div_risk = calc_diversifiable_risk(mult_df, port_list, wts)
div_risk

0.13626891569664398

In [44]:
# Risk that can't be diversified
sys_risk = port_var - div_risk
print("Systematic Risk: ", sys_risk)

Systematic Risk:  0.045941975963971204
